# SK Planner

- Managing pipes may become unmanageable depending on complexity.
- In the sequential planner, no pipes have to be defined. The planner will identify the sequential steps.
- You will need to provide the skills

## Load the required .NET packages and supporting classes

In [ ]:
#r "nuget: Microsoft.SemanticKernel, 0.17.230626.1-preview"
#r "nuget: Microsoft.Extensions.Logging.Console"
#r "nuget: dotenv.net"

using System.ComponentModel;
using System.Net.Http;
using Microsoft.Extensions.Logging;
using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.Skills.Core;
using Microsoft.SemanticKernel.SkillDefinition;
using Microsoft.SemanticKernel.Orchestration;
using Microsoft.SemanticKernel.AI.ImageGeneration;
using Microsoft.SemanticKernel.AI.ChatCompletion;
using Microsoft.SemanticKernel.Connectors.AI.OpenAI.ChatCompletion;
using Microsoft.SemanticKernel.Planning;
using Microsoft.SemanticKernel.Planning.Sequential;
using dotenv.net;
using InteractiveKernel = Microsoft.DotNet.Interactive.Kernel;

#!import Utils/ConsoleLogger.cs
#!import Utils/PlanExtensions.cs

## Load OpenAI variables from environment variables

In [ ]:
DotEnv.Load();
var deploymentName = Environment.GetEnvironmentVariable("DAVINCI_OPENAI_DEPLOYMENT_NAME");
var endpoint = Environment.GetEnvironmentVariable("DAVINCI_OPENAI_ENDPOINT");
var apiKey = Environment.GetEnvironmentVariable("DAVINCI_OPENAI_KEY");
Console.WriteLine($"Using deployment: {deploymentName} at: {endpoint} with key {apiKey.Substring(0, 5)}...");

## Create an instance of a kernel and add skills

In [ ]:
var kernel = new KernelBuilder()
    .WithLogger(ConsoleLogger.Log)
    .WithAzureTextCompletionService(deploymentName,endpoint, apiKey)
    .Build();

kernel.ImportSemanticSkillFromDirectory("./skills",
    "SummarizeSkill", "WriterSkill");

## Prepare a planner

In [ ]:
var planner = new SequentialPlanner(kernel);

## View the plan

In [ ]:
var plan = await planner.CreatePlanAsync("Write a poem about John Doe, then translate it into Italian.");

// Original plan:
// Goal: Write a poem about John Doe, then translate it into Italian.

// Steps:
// - WriterSkill.ShortPoem INPUT='John Doe is a friendly guy who likes to help others and enjoys reading books.' =>
// - WriterSkill.Translate language='Italian' INPUT='' =>

//Console.WriteLine("Original plan:");
Console.WriteLine(PlanExtensions.ToPlanString(plan));

## Execute the plan

In [ ]:
var result = await kernel.RunAsync(plan);

Console.WriteLine("Result:");
Console.WriteLine(result.Result);